In [2]:
import cv2
import os
import matplotlib.pyplot as plt
import numpy as np
from skimage import feature


In [3]:
def getBorderColor(img):
    
    border = np.asarray(img[0,:])
    border = np.concatenate((border, np.asarray(img[-1, :])))
    border = np.concatenate((border, np.asarray(img[:, 0])))
    border = np.concatenate((border, np.asarray(img[:, -1])))
    #print(border)
    #print(border)
    return np.bincount(border).argmax()

def pre_processing(img):
    #img_blurred = cv2.medianBlur(img,5)
    #img_binarized = cv2.adaptiveThreshold(img, 255, cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY, 11, 2)
    _, img_binarized = cv2.threshold(img, 120, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)    
    if(getBorderColor(img_binarized) != 0):
        img_binarized = cv2.bitwise_not(img_binarized) 
    #img_binarized = adaptiveThresh(img, t=15, div=8)
    return img_binarized

In [4]:
def load_images_from_folder(folder):
    images = []
    for filename in os.listdir(folder):
        img = cv2.imread(os.path.join(folder,filename), cv2.IMREAD_GRAYSCALE)
        if img is not None:
            images.append(pre_processing(img))
    return images

In [5]:
#follow the ration 60% 20% 20%
TRAINING_SIZE = 114
VALIDATION_SIZE = 38
TEST_SIZE = 38

TRAINING_OFFSET = TRAINING_SIZE
VALIDATION_OFFSET = TRAINING_OFFSET + VALIDATION_SIZE

training_set = []
validation_set = []
test_set = []

for i in range (1, 10):
    class_i_images = load_images_from_folder("ACdata_base/" + str(i))
    training_set.append(class_i_images[0:TRAINING_OFFSET])
    validation_set.append(class_i_images[TRAINING_OFFSET: VALIDATION_OFFSET])
    test_set.append(class_i_images[VALIDATION_OFFSET:])

In [6]:

def showImage(img):
    plt.imshow(img, cmap='gray')
    plt.show()
    #print(getBorderColor(img))
    #w, h = img.shape[:2]
    #print(cv2.countNonZero(img)/(w*h) * 100)


#arr = np.array([[1,2,3], [4,5,6], [7,8,9]])
#getBorderColor(arr)

In [7]:

# for i in range(len(test_set)):
#     for j in range(len(test_set[i])):
#         showImage(test_set[i][j])

In [8]:
def getFeaturesHOG(img):
   img = cv2.resize(img, (256, 128))
   cell_size = (32, 32)  # h x w in pixels
   block_size = (2, 2)  # h x w in cells
   nbins = 9  # number of orientation bins
   # winSize is the size of the image cropped to an multiple of the cell size
   hog = cv2.HOGDescriptor(_winSize=(img.shape[1] // cell_size[1] * cell_size[1],
                                    img.shape[0] // cell_size[0] * cell_size[0]),
                            _blockSize=(block_size[1] * cell_size[1],
                                        block_size[0] * cell_size[0]),
                            _blockStride=(cell_size[1], cell_size[0]),
                            _cellSize=(cell_size[1], cell_size[0]),
                            _nbins=nbins)

   hog_feats = hog.compute(img)
   return hog_feats.flatten() 

In [9]:
def get_X_Y_Train():
   X_Train = []
   Y_Train = []
   for i in range(len(training_set)):
      for j in range(len(training_set[i])):
        X_Train.append(getFeaturesHOG(training_set[i][j]))
        Y_Train.append(i+1)
   return (X_Train, Y_Train)

In [10]:
img = training_set[0][0]
print(len(getFeaturesHOG(img)))

756


In [11]:
# pre processing of the image (baseline estimation)
def enhanced_pre_processing(img):
    horz_proj = np.sum(img, 1)
    lb = np.argmax(horz_proj)
    avg_row_density = np.average(horz_proj)
    for inx, row_proj in enumerate(horz_proj):
        if row_proj >= avg_row_density:
            lu = inx
            break
    img[lu,:] = 255
    img[lb,:] = 255
    showImage(img)


image = load_images_from_folder("ACdata_base/1")[0]
# enhanced_pre_processing(image)


In [21]:
# feature extraction parameters
# number of cells according to paper guidlines
N = 20
# width of kernal according to paper guidlines
WIDTH = 8 
HEIGHT = int(image.shape[0] / N)
# a paramter of my selection
STRID = 4


[0.0, 0.0, 0.0, 1.55, 3.1999999999999997, 2.7624999999999997, 2.1750000000000003, 2.1625, 2.8, 3.175, 2.7875, 2.9375, 3.35, 3.025, 2.6375, 1.4625, 0.125, 1.35, 2.575, 2.7249999999999996, 3.675, 4.7625, 4.5125, 3.7125000000000004, 3.2125, 2.85, 3.0749999999999997, 3.6125, 3.925, 2.7625, 1.25, 0.8875, 0.9, 0.65, 0.525, 1.2, 1.45, 1.05, 1.15, 2.5124999999999997, 3.5625, 2.9375, 2.8375000000000004, 2.9625000000000004, 2.8749999999999996, 2.8874999999999997, 2.85, 2.9375000000000004, 1.4249999999999998, 1.1, 2.1875, 2.125, 2.0, 3.0500000000000003, 3.0625, 1.575, 1.7125, 2.325, 2.2, 1.725, 1.475, 1.7375, 1.7874999999999999, 1.325, 1.5125, 2.2875, 2.4875, 2.0875, 2.2125, 2.8874999999999997, 2.775, 2.0, 1.3624999999999998, 1.5499999999999998, 2.225, 2.325, 2.1125, 2.2375, 2.5375000000000005, 2.8625, 3.1875, 3.1625, 2.0375, 0.9625, 1.3125, 2.075, 2.3625, 2.075, 1.3875, 0.9125, 0.8875, 0.9, 1.0, 1.1874999999999998, 1.3875000000000002, 1.5125000000000002, 1.4875, 1.45, 2.675, 3.7125, 3.375, 3.262

In [ ]:
# feature f1 (density of foreground (black) pixels.)

img_height = image.shape[0]
img_width = image.shape[1]
f1 = []
# Crop out the window and calculate f1 for each cell
for r in range(img_width - 1, WIDTH - 1, -STRID):
    foreground = []
    for c in range(0, img_height, HEIGHT):
        window = image[c:c+HEIGHT-1, r-WIDTH:r]
        foreground.append(np.sum(window == 255) / (WIDTH * HEIGHT))
    f1.append(np.sum(foreground))


In [23]:
# number of transitions black/white between two consecutive cells.
f2 = []
for r in range(img_width - 1, WIDTH - 1, -STRID):
    cells = []
    #get the cells in each window
    for c in range(0, img_height, HEIGHT):
        cells.append(image[c:c+HEIGHT-1, r-WIDTH:r])
    summtion = 0
    for i in range(2,N):
        summtion += np.abs(np.sum(cells[i] == 255) -  np.sum(cells[i - 1] == 255))
    f2.append(summtion)
        


173

In [38]:
def get_center_of_mass(window):
    # calculate moments of binary image
    M = cv2.moments(window)
    # calculate x,y coordinate of center
    cX = int(M["m10"] / (M["m00"] + 1e-5))
    cY = int(M["m01"] / (M["m00"] + 1e-5))
    return cX,cY


In [42]:
# number of transitions black/white between two consecutive cells.
f3 = []
centers = []
for r in range(img_width - 1, WIDTH - 1, -STRID):
    centers.append(get_center_of_mass(image[:, r-WIDTH:r]))
f3.append(np.sqrt(np.power(centers[0][0], 2) + np.power(centers[0][1], 2)))
for i in range(1,len(centers)):
    f3.append(np.sqrt(np.power(centers[i][0] - centers[i - 1][0],
                       2) + np.power(centers[i][1] - centers[i - 1][1], 2)))

173


In [ ]:
# number of transitions black/white between two consecutive cells.
f4 = []
centers = []
for r in range(img_width - 1, WIDTH - 1, -STRID):
    centers.append(get_center_of_mass(image[:, r-WIDTH:r]))
for i in range(0, len(centers)):
    f3.append(np.sqrt(np.power(np.power(centers[i][1] - lb, 2)) / img_height)
